Notebook by Julian El-Fasih

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from tqdm import tqdm
import math
import seaborn as sns
from dateutil.parser import parse
import matplotlib.pyplot as plt

In [2]:
def is_date(string):
    
    try: 
        parse(string)
        return True

    except ValueError:
        return False

In [3]:
pd.options.mode.chained_assignment = None  # turn off warning message for chained assignment

def load_tacheometry_data():
    df_tcmt = pd.DataFrame()
    for file in tqdm(os.listdir("../Data/Tacheometry")):
        
        # extract objectid and define header
        objectid = file[:7]
        if objectid == 'PRG0101':
            header = [2,3,4,5]
        else:
            header = [1,2,3,4]
            
        # read sheet with deformation time series
        df = pd.read_excel("../Data/Tacheometry" + '/' + file, sheet_name="Deformatiemeetpunten", header=header)
        df = df.fillna("n.g.")
        points = df.loc[:, "Nulmeting"].iloc[:,0].rename("")
        
        # initiate dataframe with points as index
        df_sub = pd.DataFrame(index=points)
        
        # extract coordinates of points
        df_coords = pd.read_excel("../Data/Tacheometry" + '/' + file, sheet_name="Nulmeting", header=[1,2,3,4])

        # fix for inconsistency in label name 'coordinaten'/'coördinaten'
        if objectid in ['BRG0602', 'DYG0102', 'KZG0201', 'KZG0301', 'KZG0901', 'LEG0402', 'LYG0603', \
                        'LYG1001', 'OAW0602', 'OAW0701', 'OAW0702', 'OEV2491', 'SIG0605', 'SIN0501']:
            coords = df_coords.iloc[:, (df_coords.columns.get_level_values(2)=='RD-Coordinaten (m)') | \
                                    (df_coords.columns.get_level_values(2)=='RD-Coördinaten (m)')].iloc[:,[0,1]]
            coord_index = df_coords.iloc[:, df_coords.columns.get_level_values(2) == \
                                         'Deformatiemeetpunten'].iloc[:,0].rename("")
        else:
            coords = df_coords.iloc[:, (df_coords.columns.get_level_values(2)=='RD-Coordinaten (m)') | \
                                    (df_coords.columns.get_level_values(2)=='RD-Coördinaten (m)')].iloc[:,[-2,-1]]
            coord_index = df_coords.iloc[:, df_coords.columns.get_level_values(2) == \
                                         'Deformatiemeetpunten'].iloc[:,-1].rename("")

        coords.columns = coords.columns.droplevel([0,1,2])
        coords = coords.set_index(coord_index).reset_index()
        coords = coords.merge(pd.DataFrame(points), on= '', how='right').set_index('')

        # extract dates of measurement
        # for missing dates, took date between preceding and following date, inserted in excel file
        dates = []
        for item in df.columns.get_level_values(1):
            if isinstance(item, datetime.date):
                dates.append(item)
            elif is_date(item):
                dates.append(parse(item))
        
        dates = sorted(list(set(dates)))[1:]

        # extract deformations
        deformations = df.iloc[:, df.columns.get_level_values(1)=='rel tov nulmeting']
        deformations.columns = deformations.columns.droplevel([0, 1, 2])
        
        for i in range(0, len(deformations.columns), 3):
            deforms = []
            
            # turn deformations into tuples (dX, dY, dZ)
            for j, deform in enumerate(deformations.iloc[:,[i,i+1,i+2]].values):
                deforms.append(tuple(deform))
  
            df_sub[f'date{int(i/3)+1}'] = deforms

        # add dates and point coordinates to the dataframe
        df_sub.columns = dates
        df_sub['objectid'] = objectid
        df_sub['X_coordinate'] = coords['X']
        df_sub['Y_coordinate'] = coords['Y']
        df_tcmt = pd.concat([df_tcmt, df_sub])
    
    # finalize dataframe
    sorted_dates = [date for date in df_tcmt.columns if isinstance(date, datetime.date)]
    other_columns = [col for col in df_tcmt.columns if isinstance(col, datetime.date) == False]
    col_order = other_columns + sorted(sorted_dates)
    df_tcmt = df_tcmt.reindex(col_order, axis=1)
    df_tcmt = df_tcmt.reset_index(drop=True)
    
    return df_tcmt

In [4]:
df_tcmt = load_tacheometry_data()

 21%|████████████████▋                                                                | 26/126 [00:10<00:47,  2.10it/s]C:\Users\julia\AppData\Local\Temp/ipykernel_14912/3703224126.py:38: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  coords = coords.merge(pd.DataFrame(points), on= '', how='right').set_index('')
 21%|█████████████████▎                                                               | 27/126 [00:11<00:53,  1.83it/s]C:\Users\julia\AppData\Local\Temp/ipykernel_14912/3703224126.py:38: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  coords = coords.merge(pd.DataFrame(points), on= '', how='right').set_index('')
 24%|███████████████████▎                                                             | 30/126 [00:12<00:47,  2.03it/s]C:\Users\julia\AppData\Local\Temp/ipykernel_1

In [5]:
# remove rows with missing values for coordinates
X_missing = df_tcmt['X_coordinate'][df_tcmt['X_coordinate'].apply(lambda x: not isinstance(x, (int, float, complex)))]
Y_missing = df_tcmt['Y_coordinate'][df_tcmt['Y_coordinate'].apply(lambda x: not isinstance(x, (int, float, complex)))]
missing_indices = pd.concat([X_missing, Y_missing], axis=1).index
df_tcmt = df_tcmt.drop(missing_indices, axis=0)

In [6]:
# get only coordinates for visualistation purposes
df_tcmt_vis = df_tcmt[['objectid', 'X_coordinate', 'Y_coordinate']]

In [7]:
df_tcmt_vis.to_csv('../Data/Tacheometry_vis.csv', index=True)

In [8]:
df_tcmt_raw = df_tcmt[['objectid', 'X_coordinate', 'Y_coordinate']]
df_tcmt_split = df_tcmt[['objectid', 'X_coordinate', 'Y_coordinate']]

# runs 552 iterations (2 minutes approx)
for i, col in tqdm(enumerate(df_tcmt[df_tcmt.columns[3:]])):
    
    # turn NaN into tuple (nan, nan, nan)
    df_tcmt[col] = df_tcmt[col].apply(lambda x: (np.nan, np.nan, np.nan) if x is np.nan else x)

    # get column names in format X0, Y0, Z0, X1, Y1, Z1 etc.
    df_tcmt_split[f'X{int(i)}'], df_tcmt_split[f'Y{int(i)}'], df_tcmt_split[f'Z{int(i)}'] = zip(*df_tcmt[col])
    df_tcmt_split = df_tcmt_split.copy()
    
    # get column names in format X{date}, Y{date}, Z{date}
    df_tcmt_raw[f'X{df_tcmt.columns[3+i]}'], df_tcmt_raw[f'Y{df_tcmt.columns[3+i]}'], \
                                                df_tcmt_raw[f'Z{df_tcmt.columns[3+i]}'] = zip(*df_tcmt[col])
    df_tcmt_raw = df_tcmt_raw.copy()

552it [02:18,  3.98it/s]


In [9]:
# check which non numeric and non-nan values are in df
non_numerics = []

for col in df_tcmt_split.columns[3:]:
    non_numerics += [value for value in df_tcmt_split[col].unique() if not isinstance(value, (int, float, complex))]

non_numerics = set(non_numerics)
print(non_numerics)

{'ng', 'n.g', 'g.v.m', 'n.v.t..', 'n.g.', 'NULM', 'n.v.t.', 'vervallen', 'n,g,'}


In [10]:
# replace those with nan
df_tcmt_split = df_tcmt_split.replace(non_numerics, np.NaN)
df_tcmt_raw = df_tcmt_raw.replace(non_numerics, np.NaN)

In [11]:
# remove rows with missing values for coordinates
missing_X = df_tcmt_split[df_tcmt_split['X_coordinate'].isna()]
missing_Y = df_tcmt_split[df_tcmt_split['Y_coordinate'].isna()]
missing_indices = pd.concat([missing_X, missing_Y], axis=1).index
df_tcmt_split = df_tcmt_split.drop(missing_indices, axis=0)
df_tcmt_raw = df_tcmt_raw.drop(missing_indices, axis=0)

In [12]:
df_tcmt_split.to_csv('../Data/Tacheometry.csv', index=True)

In [13]:
df_tcmt_raw.to_csv('../Data/Tacheometry_raw.csv', index=True)